# LLM Compiler Agent

Full credits to the [source repo for LLMCompiler](https://github.com/SqueezeAILab/LLMCompiler).

A lot of our implementation was lifted from this repo (and adapted with LlamaIndex modules).

LLM Compiler agent notebook

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [4]:
from llama_index.agent import AgentRunner
from llama_index.agent.llm_compiler.step import LLMCompilerAgentWorker

In [5]:
agent_worker = LLMCompilerAgentWorker.from_tools(
    tools,
    llm=OpenAI(model="gpt-4")
)
agent = AgentRunner(agent_worker)

PREFIX: Given a user query, create a plan to solve it with the utmost parallelizability. Each plan should comprise an action from the following 3 types:
1. multiply(a: int, b: int) -> int
Multiple two integers and returns the result integer
2. add(a: int, b: int) -> int
Add two integers and returns the result integer
3. join():
 - Collects and combines results from prior actions.
 - A LLM agent is called upon invoking join to either finalize the user query or wait until the plans are executed.
 - join should always be the last action in the plan, and will be called in two scenarios:
   (a) if the answer can be determined by gathering the outputs from tasks to generate the final response.
   (b) if the answer cannot be determined in the planning phase before you execute the plans. 

Guidelines:
 - Each action described above contains input/output types and description.
    - You must strictly adhere to the input and output types for each action.
    - The action descriptions contain the

In [6]:
response = agent.chat("What is (121 * 3) + 42?")

text Plan:
1. multiply(121, 3) -> 363
2. add($1, 42) -> 405
3. join()
<END_OF_PLAN>
matches [('', '1', 'multiply', '121, 3', ''), ('', '2', 'add', '$1, 42', ''), ('', '3', 'join', '', '')]
results [LLMCompilerParseResult(thought='', idx=1, tool_name='multiply', args='121, 3'), LLMCompilerParseResult(thought='', idx=2, tool_name='add', args='$1, 42'), LLMCompilerParseResult(thought='', idx=3, tool_name='join', args='')]


In [7]:
response

AgentChatResponse(response='405', sources=[], source_nodes=[])